In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cp /content/drive/MyDrive/VRD-IU/train_data.pkl .

In [3]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import pickle
from transformers import AutoTokenizer
import torch
class CompTextDataset(Dataset):
    def __init__(self, pickle_file):
        super().__init__()
        with open(pickle_file, 'rb') as file:
            data = pickle.load(file)
        self.components = []
        for k in data.keys():
            for comp in data[k]['components']:
                if comp['bbox'] == [0.0, 0.0, 0.0, 0.0]:
                  continue
                self.components.append(comp)

    def __len__(self):
        return len(self.components)

    def __getitem__(self, index):
        comp = self.components[index]
        try:
            text = comp['text']
        except:
            text = comp['category']
        return text, comp['object_id']

In [4]:
train_dataset = CompTextDataset('train_data.pkl')

In [5]:
from transformers import AutoModel
model = AutoModel.from_pretrained('bert-base-uncased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using: {device}")
model.to(device)
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Using: cuda


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [6]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
from tqdm import tqdm
import os
import pandas as pd
def extract_features(dataloader, feature_path):
  features_list = []
  object_id_list = []
  with torch.no_grad():
      for texts, object_ids in tqdm(dataloader):
          text_inputs = tokenizer(texts, return_tensors="pt",padding=True, truncation=True).to(device)
          outputs = model(**text_inputs)
          features = outputs[0][:,0].detach().cpu()
          features_list.extend(features.tolist())
          object_id_list.extend(object_ids.tolist())
      pd.DataFrame({"id":object_id_list,"features":features_list}).to_pickle(feature_path)

In [8]:
train_dataloader = DataLoader(train_dataset,batch_size=256, num_workers=2)

In [9]:
extract_features(train_dataloader,  'train_textual_features.pkl')
print("Extraction completed for training set!")

100%|██████████| 170/170 [22:01<00:00,  7.77s/it]

Extraction completed for training set!


In [10]:
!du -sh train_textual_features

32G	train_textual_features


In [13]:
!zip -r train_textual_features.zip train_textual_features

Streaming output truncated to the last 5000 lines.
  adding: train_textual_features/63252.pt (deflated 62%)
  adding: train_textual_features/51753.pt (deflated 12%)
  adding: train_textual_features/57040.pt (deflated 14%)
  adding: train_textual_features/62013.pt (deflated 32%)
  adding: train_textual_features/59975.pt (deflated 11%)
  adding: train_textual_features/51439.pt (deflated 14%)
  adding: train_textual_features/37693.pt (deflated 12%)
  adding: train_textual_features/49985.pt (deflated 10%)
  adding: train_textual_features/55010.pt (deflated 16%)
  adding: train_textual_features/42076.pt (deflated 16%)
  adding: train_textual_features/56681.pt (deflated 44%)
  adding: train_textual_features/18357.pt (deflated 10%)
  adding: train_textual_features/39518.pt (deflated 12%)
  adding: train_textual_features/51392.pt (deflated 17%)
  adding: train_textual_features/8108.pt (deflated 26%)
  adding: train_textual_features/59321.pt (deflated 24%)
  adding: train_textual_features/48373

In [ ]:
!cp -r train_textual_features /content/drive/MyDrive/VRD-IU

Streaming output truncated to the last 5000 lines.
  adding: train_textual_features/30041.pt (deflated 37%)
  adding: train_textual_features/16562.pt (deflated 21%)
  adding: train_textual_features/4810.pt (deflated 12%)
  adding: train_textual_features/42036.pt (deflated 14%)
  adding: train_textual_features/58176.pt (deflated 11%)
  adding: train_textual_features/12654.pt (deflated 15%)
  adding: train_textual_features/50584.pt (deflated 15%)
  adding: train_textual_features/39619.pt (deflated 12%)
  adding: train_textual_features/42930.pt (deflated 9%)
  adding: train_textual_features/60442.pt (deflated 58%)
  adding: train_textual_features/51323.pt (deflated 14%)
  adding: train_textual_features/15985.pt (deflated 15%)
  adding: train_textual_features/13406.pt (deflated 31%)
  adding: train_textual_features/53595.pt (deflated 14%)
  adding: train_textual_features/2598.pt (deflated 20%)
  adding: train_textual_features/18729.pt (deflated 10%)
  adding: train_textual_features/46582.p